In [2]:
applied_for_JRIDs = ['R0001165', 'R0000078', 'R0000364', '01222651', 
                     '01221549', '01219721', '01222378', '01222260',
                     '01222212', 'R0001799', 'R0001743', 'R0002022']
potential_JRIDs = ['01219721', '01222260', '01222378', '01222651', 
                   'R0001743', 'R0001799', 'R0002022']
apply_for_list = list(set(potential_JRIDs) - set(applied_for_JRIDs))
print(apply_for_list)
len(apply_for_list)

[]


0

In [3]:
import pickle

dir = u"../Administrivia/Job Hunting/"
if len(apply_for_list) == 0:
    with open(dir + 'applied_for_JRIDs.pickle', 'wb') as handle:
        pickle.dump(applied_for_JRIDs, handle, pickle.HIGHEST_PROTOCOL)

In [1]:
%matplotlib inline
import os
import pandas as pd

#ValueError: time data '3/2/2017' does not match format '%Y-%m-%d %H:%M:%S'
date_format = "%m/%d/%Y"
date_column_list = ['Recruiting Start Date']
date_parser = lambda x: None if x == 'nan' else pd.datetime.strptime(str(x), date_format)
dir = u"../Administrivia/Job Hunting/"

encoding = 'latin1'
#encoding = 'iso8859-1'
#encoding = 'utf-8'
column_list = []

In [2]:
for root, dirs, files in os.walk(dir):
    #path = root.split(os.sep)
    #print((len(path)-1) * '---', os.path.basename(root))
    for file in files:
        #print(len(path) * '---', file)
        if file.endswith(".csv"):
            file_name = os.path.join(dir, file)
            if os.path.isfile(file_name):
                df = pd.read_csv(file_name, parse_dates=date_column_list, date_parser=date_parser, encoding=encoding)
                column_list = list(set(column_list) | set(df.columns.tolist()))

In [3]:
everybody_df = pd.DataFrame([], columns=column_list)

for root, dirs, files in os.walk(dir):
    for file in files:
        if file.endswith(".csv"):
            file_name = os.path.join(dir, file)
            if os.path.isfile(file_name):
                df = pd.read_csv(file_name, parse_dates=date_column_list, date_parser=date_parser, encoding=encoding)
                everybody_df = pd.concat([everybody_df, df])

In [4]:
column_list

['Location',
 'Management Level',
 'Job Requisition ID',
 'Job Posting',
 'Required Clearance',
 'Clearance Agency',
 'Job Profile',
 'Account Group',
 'Job Description',
 'IMT',
 'Cluster',
 'Resource Manager',
 'Primary Recruiter',
 'Group',
 'Job Requisition Type',
 'Recruiting Start Date',
 'Hiring Manager',
 'Job Family']

In [11]:
everybody_df['Job Profile'].unique()

array(['IT Project Mgr', 'SD Bus Sys Analyst Mid', 'SD Systems Analyst Sr',
       'Network Engineer Sr', 'Learning & Dev Consult Sr',
       'SD Test Engineer', 'Cyber Threat Analyst', 'IP Systems Admin',
       'SD Bus Sys Analyst Sr', 'SD Web Designer Sr',
       'Project Analyst Mid', 'Software Engineer', 'SD Project Analyst Sr',
       'Geospatial Analyst Mid', 'Bus Ops Spec Mid', 'Bus Ops Spec Sr',
       'Web Developer Mid', 'Security Admin Mid', 'Info Secur Eng Mid',
       'Network Ops Sr Mgr', 'Cyber Sec Pol & Comp Analyst',
       'Cyber Sec Pol & Comp Analyst Sr', 'Info Secur Eng Sr',
       'Foreign Policy Analyst Lead', 'SD Test Engineer Mid',
       'Network Systems Design Eng Sr', 'Comm-Cyber Sec Pol&Comp Anl Sr',
       'Language Analyst Mid', 'Perf Consultant Mid', 'Systems Admin Lead',
       'Comm-Cyber Sec Pol&Comp An Mid', 'Subcontract Admin Mid',
       'Network Engineer Mid', 'Fin Sys Analyst Mid', 'SD Bus Sys Analyst',
       'A/R Bill/Briefing Spec', 'Systems 

In [ ]:
def f(row):
    derived_name = str(row['StudentLastName']) + ", " + str(row['StudentFirstName']) + " (" + str(row['derived_event']) + ")"
    derived_name = derived_name.upper()
    
    return derived_name

t0 = time.time()
everybody_df['derived_name'] = everybody_df.apply(lambda row: f(row), axis=1)
t1 = time.time()
print(t1-t0)

In [10]:
match = everybody_df['Required Clearance'].isin(['None', 'Secret', 'Confidential'])

array([True, False], dtype=object)

In [9]:
match = everybody_df['Job Requisition ID'].isin(apply_for_list)
everybody_df[match]['Location'].unique()

array(['USA, VA, Alexandria (6361 Walker Ln)',
       'USA, DC, Washington (901 15th St NW)',
       'USA, CA, San Diego (2841 Rendova Rd)',
       'USA, NC, Fayetteville (4200 Morganton Rd)',
       'USA, MA, Boston (50 Milk St)'], dtype=object)

In [ ]:
rows_list = []
for column_name in blank_columns:
    
    # Get input row in dictionary format; key = col_name
    row_dict = {}
    row_dict['column_name'] = column_name
    row_dict['count_blanks'] = everybody_df[column_name].isnull().sum()
    row_dict['count_uniques'] = len(everybody_df[column_name].unique())
    
    rows_list.append(row_dict)

blank_ranking_df = pd.DataFrame(rows_list, columns=['column_name', 'count_blanks', 'count_uniques'])